### DICOM2Nifti
Some dicom files in SPIROMICS were reported to have private information left to be unremoved. The dicom files need to be removed after original files are converted to Nifti format. The code here is used to convert dicom files to nifti files with name tags removed.<br/>
\------------------------------------------------------------------------<br/>
Author:   Chunrui Zou<br/>
Date:  07/14/2020


In [ ]:
## if itk is not installed
!pip install itk

In [103]:
import itk
import numpy as np
import os

In [13]:
os.getcwd()

'C:\\Users\\chunr\\OneDrive - University of Iowa\\SPIROMICS_DICOM2Nifti'

In [104]:
PixelType = itk.ctype("short")
image_t = itk.Image[PixelType,3]
reader = itk.ImageSeriesReader[image_t].New()
dicomIO = itk.GDCMImageIO.New()
dicomFN = itk.GDCMSeriesFileNames.New()
reader.SetImageIO(dicomIO)

In [105]:
dicomFN.SetUseSeriesDetails(True) # Use more information to identify individual 3D image
dicomFN.SetDirectory("./SampleDicom/unrarred/")

In [106]:
uids = dicomFN.GetSeriesUIDs()
fnames = dicomFN.GetFileNames(uids[0])
reader.SetFileNames(fnames)
reader.Update()
image = reader.GetOutput()
metad = dicomIO.GetMetaDataDictionary()

In [169]:
something=metad.Get('0008|0000')


'MetaDataObject (0000021A9C768220)\n  RTTI typeinfo:   class itk::MetaDataObject<class std::basic_string<char,struct std::char_traits<char>,class std::allocator<char> > >\n  Reference Count: 5\n'

In [162]:
dir(metad)

['Clear',
 'Erase',
 'Get',
 'GetKeys',
 'HasKey',
 'Print',
 'Set',
 'Swap',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__swig_destroy__',
 '__weakref__',
 'this',
 'thisown']

In [ ]:
dicomIO.SetMetaDataDictionary()

In [160]:
labelId=""
dicomIO.GetLabelFromTag('0008|0000',labelId)
dicomIO.GetValueFromTag('0008|0000',labelId)

[True, '614']

In [161]:
metad['0008|0000']

'614'

In [129]:
reader.GetMetaDataDictionaryArray()

TypeError: in method 'itkMetaDataDictionary_Get', argument 2 of type 'std::string const &'

In [134]:
itk.MetaDataObject.New()

<itkMetaDataObjectPython.itkMetaDataObjectD; proxy of <Swig Object of type 'itkMetaDataObjectD *' at 0x0000021A82979510> >

In [ ]:
nifWriter = itk.ImageFileWriter[image_t].New()
nifti_io = itk.NiftiImageIO.New()
nifWriter.SetFileName("test.nii.gz")
nifWriter.SetInput(image)
nifWriter.SetImageIO(nifti_io)
nifWriter.Update()

In [108]:
lab2Keep = ('0008|0000','0008|0008','0008|0012','0008|0013','0008|0020','0008|0021',
'0008|0022','0008|0023','0008|0030','0008|0031','0008|0032','0008|0033',
'0008|0060','0008|1030','0008|103e','0010|0000','0010|0010','0010|0020',
'0018|0000','0018|0050','0020|0000','0020|000d','0020|000e','0028|0000',
'0028|0010','0028|0011','0028|0030','0028|0101')
#GenericGroupLength,ImageType, InstanceCreationDate, InstanceCreationTime,
#StudyDate, SeriesDate,AcquisionDate,ContentDate,StudyTime,SeriesTime,
#AcquisionTime,ContentTime,Modality,StudyDescription,SeriesDescription,
#GenericGroupLength,PatientName, PatientID,GenericGroupLength,SliceThickness,
#GenericGroupLength,StudyInstanceUID,SeriesInstanceUID,GenericGroupLength,
#Rows,Columns,PixelSpacing,BitsStored

for label in metad.GetKeys():
    if label in lab2Keep:
        continue
    else:
        metad.Erase(label)

In [109]:
image_2d = itk.Image[PixelType,2]
dcmWriter = itk.ImageSeriesWriter[image_t,image_2d].New()
dicom_io = itk.GDCMImageIO.New()
dcmWriter.SetImageIO(dicom_io)
dcmWriter.SetInput(reader.GetOutput())
numSlice=reader.GetOutput().GetLargestPossibleRegion().GetSize()[2]
fnames = itk.NumericSeriesFileNames.New()
fnames.SetSeriesFormat("./test2/%d.dcm")
fnames.SetStartIndex(1)
fnames.SetEndIndex(numSlice)
dcmWriter.SetFileNames(fnames.GetFileNames())
#dcmWriter.SetFileName("./test.dcm")
#dcmWriter.UseInputMetaDataDictionaryOff()
dcmWriter.Update()

In [124]:
nameGenerator = itk.GDCMSeriesFileNames.New()
nameGenerator.SetUseSeriesDetails(True)
nameGenerator.SetMetaDataDictionary(metad)
nameGenerator.SetDirectory("./test2")

image_2d = itk.Image[PixelType,2]
dcmWriter = itk.ImageSeriesWriter[image_t,image_2d].New()
dicom_io = itk.GDCMImageIO.New()
dcmWriter.SetImageIO(dicom_io)
dcmWriter.SetFileNames(nameGenerator.GetOutputFileNames())
dcmWriter.SetInput(image)
dcmWriter.Update()

In [125]:
nameGenerator.GetOutputFileNames()

()